In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # setting ignore as a parameter and further adding category
from whispr import *
import os


In [18]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 3
source_plate_type = '384PP_AQ_BP' 

os.getcwd()
folder = os.getcwd() + '/Experiments/gfp_demo/'
sp_plasmids_file = folder + 'plasmids_sp_230809.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0, engine='openpyxl')
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

layout_genex_file = folder + 'genex_pl_384.csv'
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = folder + 'genex_mt_230809.xlsx'
#mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)
mt_genex = pd.read_excel(mt_genex_file, index_col = 0, dtype = str, engine = 'openpyxl').fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=folder+'plasmids_sp_updated.xlsx')
protocol_genex_df = protocol_genex_df[0]


Volumes used from each well for this protocol:
{'1_A1': 1.7, '1_P4': 0.575, '1_M4': 0.725}


In [19]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


# change this#
txtl_well = 'E17'
##

for label, dwell in rxn_loc.items():
    for d in dwell:
        print(label)
        txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
        row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                    'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
        protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv(folder+'protocol_genex.csv',index = False)
print('TXTL needed: '+str(protocol_genex_df[protocol_genex_df['Source Well'] == txtl_well]['Transfer Volume'].sum()/1000))
# carry out gene expression reaction

rxn_1
rxn_2
rxn_3
rxn_4
TXTL needed: 9.0


In [5]:
# dilution. everything gets Tris, for 20x -> 57uL

##### doing it by hand if not too many reactions #####

# protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
# source_plate_type = '6RES_AQ_B' 
# tris_well = 'A1'
# tris_vol = 57*1000

# for dwell in protocol_genex_df['Destination Well'].unique():
#     row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
#                                 'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
#     protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

# protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [6]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [5]:
folder = os.getcwd() + '/Experiments/230503/'


In [7]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = folder + 'buffers_sp_230809.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0, engine = 'openpyxl')
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
#sp_genex = sp_from_layout(layout_genex, 60)
layout_genex_384_file = folder + 'genex_pl_384.csv'
layout_genex_384 = pd.read_csv(layout_genex_384_file, index_col = 0, dtype = str)
sp_genex = sp_from_layout(layout_genex_384, 60)


# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = folder + 'biosyn_mt_230809.xlsx'
mt_biosyn = pd.read_excel(mt_biosyn_file, index_col = 0, dtype = str, engine = 'openpyxl').fillna(0)
mt_biosyn = mt_biosyn[~mt_biosyn.index.isna()]

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
layout_biosyn1_file = folder + 'biosyn_pl.csv' 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
layout_biosyn1 = layout_biosyn1[~layout_biosyn1.index.isna()]

# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol= folder +'combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

Volumes used from each well for this protocol:
{'1_B1': 13.0, '1_C1': 2.0, '1_D1': 13.0, '1_E1': 30.0, '1_F1': 6.0, '1_H1': 12.0, '1_J1': 5.0, '1_K1': 6.0, '1_L1': 2.0, '1_N1': 4.0, '1_O1': 1.2, '1_P1': 2.0, '1_B3': 2.5, '1_P3': 8.0, '1_O3': 9.0, '1_M3': 9.0, '1_M4': 1.0, '1_G4': 18.75, '2_A2': 615.55, '3_C16': 5.0, '3_C17': 5.0, '3_C18': 5.0, '3_C19': 5.0, '3_C20': 5.0, '3_C21': 5.0, '3_C22': 5.0, '3_C23': 5.0, '3_C24': 5.0, '3_D21': 5.0, '3_D22': 5.0, '3_D23': 5.0, '3_D24': 35.0}


In [8]:
protocol_biosyn_dfs[0].to_csv(folder+'protocol_biosyn_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv(folder+'protocol_biosyn_6RES.csv',index = False)